In [41]:
from sklearn.datasets import make_blobs
from sklearn.decomposition import PCA
from lib.CF_De.cfde import CF_Descent
from lib.cf_a.cfae import CF_Ascent
from lib.eval.eval_metrics import *
import numpy as np
import random
from lib.eval.tools import center_prediction

%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Evaluation of custom algorithms specifications

In this workspace we test what parameters works best for CFDE and CFAE.

In [42]:
X, y, centers = make_blobs(centers=4, cluster_std=8, n_features=30, n_samples=1500, return_centers=True)

pca = PCA(n_components=2)
embed_X = pca.fit_transform(X)



## CFDE

## Immutable ratio

In [ ]:
values = np.linspace(0.0,1.0,num=31)

n = 10
target = 1.0
pred = 1.0
while pred == target:
    index = random.randint(0,len(X))
    new_instance = X[index]
    pred = y[index]

best = -1
best_metric = -float('inf')
for ratio in values:
    metric = 0
    for i in range(n):
        x, cf, h = CF_Descent(X,y,1.0,centers,instance_index=index,new_immutable_ratio=ratio)
        x = x.reshape(1,-1)
        cf = cf.reshape(1,-1)

        metric += cf_similarity(x, cf)[0] * cf_minimality(cf, x) * cf_plausibility(cf, target, X, y)[0]
    metric = metric / n
    if metric > best_metric:
        best_metric = metric
        best = ratio

print("Best immutable ratio is:", best)

Features that can be changed count:  29
Amount of changes:  496
Number of changed features: 12
Features that can be changed count:  29
Amount of changes:  537
Number of changed features: 12
Features that can be changed count:  29
Amount of changes:  422
Number of changed features: 10
Features that can be changed count:  29
Amount of changes:  391
Number of changed features: 10
Features that can be changed count:  29
Amount of changes:  478
Number of changed features: 11
Features that can be changed count:  29
Amount of changes:  531
Number of changed features: 13
Features that can be changed count:  29
Amount of changes:  495
Number of changed features: 11
Features that can be changed count:  29
Amount of changes:  501
Number of changed features: 12
Features that can be changed count:  29
Amount of changes:  495
Number of changed features: 12
Features that can be changed count:  29
Amount of changes:  514
Number of changed features: 13
Features that can be changed count:  28
Amount of 

KeyboardInterrupt: 

## CFAE

## CFDE vs CFAE

In [47]:
it = 10
n = 10
target = 1.0
pred = 1.0
while pred == target:
    index = random.randint(0,len(X))
    new_instance = X[index]
    pred = y[index]

count_de = 0
count_ae = 0
for _ in range(it):
    metric_de = 0
    metric_ae = 0
    for i in range(n):
        x1, cf1, h1 = CF_Descent(X,y,1.0,centers,instance_index=index)
        x2, cf2, h2 = CF_Ascent(X,y,1.0,centers,instance_index=index)

        x1 = x1.reshape(1,-1)
        cf1 = cf1.reshape(1,-1)

        x2 = x2.reshape(1,-1)
        cf2 = cf2.reshape(1,-1)

        metric_de += cf_similarity(x1, cf1)[0]
        metric_ae += cf_similarity(x2, cf2)[0]
    metric_de = metric_de / n
    metric_ae = metric_ae / n
    if metric_de > metric_ae:
        count_de += 1
    else:
        count_ae += 1

print("cfde score:", count_de/it)
print("cfae score:", count_ae/it)

Features that can be changed count:  19
Amount of changes:  80
Number of changed features: 10
Amount of changes:  34
Number of changed features: 5
Features that can be changed count:  19
Amount of changes:  99
Number of changed features: 11
Amount of changes:  42
Number of changed features: 6
Features that can be changed count:  19
Amount of changes:  103
Number of changed features: 11
Amount of changes:  36
Number of changed features: 6
Features that can be changed count:  19
Amount of changes:  103
Number of changed features: 10
Amount of changes:  40
Number of changed features: 6
Features that can be changed count:  19
Amount of changes:  63
Number of changed features: 10
Amount of changes:  35
Number of changed features: 5
Features that can be changed count:  19
Amount of changes:  74
Number of changed features: 10
Amount of changes:  35
Number of changed features: 6
Features that can be changed count:  19
Amount of changes:  71
Number of changed features: 10
Amount of changes:  39